In [231]:
import os 
import pandas as pd
import numpy as np

path = os.getcwd()

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

import seaborn as sns
import matplotlib.pyplot as plt

In [232]:
path = 'C:\\Users\\sunil\\Projects\\Dockship\\segmind_grand_ai_challenge_2021-dataset'

In [233]:
train = pd.read_csv(path + "\\dataset\\TRAIN.csv")
test = pd.read_csv(path + "\\dataset\\TEST.csv")
ss = pd.read_csv(path + "\\dataset\\sample_submission.csv")
shop = pd.read_csv(path + "\\dataset\\store.csv")

C:\Users\sunil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [234]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
train['dayofweek'] = train['Date'].dt.dayofweek
train['week'] = train['Date'].dt.week

test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['day'] = test['Date'].dt.day
test['dayofweek'] = test['Date'].dt.dayofweek
test['week'] = test['Date'].dt.week

train['StateHoliday'].replace({0 : 0, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)
test['StateHoliday'].replace({0 : 0, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)

In [235]:
# Merging

store_type = dict(zip(shop['Store'], shop['StoreType']))
store_ass = dict(zip(shop['Store'], shop['Assortment']))
store_com = dict(zip(shop['Store'], shop['CompetitionDistance']))
store_month = dict(zip(shop['Store'], shop['CompetitionOpenSinceMonth']))
store_year = dict(zip(shop['Store'], shop['CompetitionOpenSinceYear']))
store_p2 = dict(zip(shop['Store'], shop['Promo2']))
store_pweek = dict(zip(shop['Store'], shop['Promo2SinceWeek']))
store_pyear = dict(zip(shop['Store'], shop['Promo2SinceYear']))
store_pi = dict(zip(shop['Store'], shop['PromoInterval']))

train['Store_Type'] = train['Store'].map(store_type)
train['Store_Assortment'] = train['Store'].map(store_ass)
train['Store_distance'] = train['Store'].map(store_com)
train['Store_CompetitionOpenSinceMonth'] = train['Store'].map(store_month)
train['Store_CompetitionOpenSinceYear'] = train['Store'].map(store_year)
train['Store_Promo2'] = train['Store'].map(store_p2)
train['Store_Promo2SinceWeek'] = train['Store'].map(store_pweek)
train['Store_Promo2SinceYear'] = train['Store'].map(store_pyear)
train['Store_PromoInterval'] = train['Store'].map(store_pi)


test['Store_Type'] = test['Store'].map(store_type)
test['Store_Assortment'] = test['Store'].map(store_ass)
test['Store_distance'] = test['Store'].map(store_com)
test['Store_CompetitionOpenSinceMonth'] = test['Store'].map(store_month)
test['Store_CompetitionOpenSinceYear'] = test['Store'].map(store_year)
test['Store_Promo2'] = test['Store'].map(store_p2)
test['Store_Promo2SinceWeek'] = test['Store'].map(store_pweek)
test['Store_Promo2SinceYear'] = test['Store'].map(store_pyear)
test['Store_PromoInterval'] = test['Store'].map(store_pi)


train['Store_Assortment'].unique()
train['Store_Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)
test['Store_Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)


a = [0, 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec']
b = [0, 1, 2, 3]

train['Store_PromoInterval'].replace(dict(zip(a, b)), inplace = True)
test['Store_PromoInterval'].replace(dict(zip(a, b)), inplace = True)

train['Store_Type'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)
test['Store_Type'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)

train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

In [236]:
# Changing Datatype

int8_cols = ['DayOfWeek', 'Open', 'Promo','StateHoliday', 'SchoolHoliday', 
'month', 'day', 'dayofweek','week', 'Store_Type', 'Store_Assortment',
'Store_CompetitionOpenSinceMonth','Store_Promo2', 
'Store_Promo2SinceWeek','Store_PromoInterval']

int16_cols = ['Customers', 'year', 'Store', 'Store_CompetitionOpenSinceYear', 'Store_Promo2SinceYear']

int32_cols = ['Sales', 'Store_distance']

def int8(x):
    train[x] = train[x].astype('int8')
    
    test[x] = test[x].astype('int8')

def int16(x):
    train[x] = train[x].astype('int16')
    if x != 'Customers':
        test[x] = test[x].astype('int16')

def int32(x):
    train[x] = train[x].astype('int32')
    if x != 'Sales':
        test[x] = test[x].astype('int32')

for col in int8_cols:
    int8(col)

for col in int16_cols:
    int16(col)

for col in int32_cols:
    int32(col)

In [193]:
# # Dropping Samples with Open=0

# index = train[ train['Open'] == 0].index
# train.drop(index, inplace = True, axis = 0)

In [237]:
train['Sales'] = np.sqrt(train['Sales'])

In [238]:
target = 'Sales'
date = 'Date'
customer = 'Customers'

cat_cols = ['Store', 'DayOfWeek', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'year', 'month', 'day', 'dayofweek',
       'week', 'Store_Type', 'Store_Assortment',
       'Store_CompetitionOpenSinceMonth', 'Store_CompetitionOpenSinceYear',
       'Store_Promo2', 'Store_Promo2SinceWeek', 'Store_Promo2SinceYear',
       'Store_PromoInterval']

features = [col for col in train.columns if col not in [target, date, customer]]

In [239]:
trn, val = train_test_split(train, test_size = 0.2, random_state = 1999)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test[features]

In [240]:
def join(train_, test_):
    df = pd.concat([train_, test_], axis = 0).reset_index(drop=True)
    return df

def split(df):
    train_, test_ = df[:train.shape[0]], df[train.shape[0]:].reset_index(drop=True)
    return train_, test_

In [241]:
# store's mean, max, min sale

mean = train.groupby(['Store'])['Sales'].mean()
median = train.groupby(['Store'])['Sales'].median()
max_ = train.groupby(['Store'])['Sales'].max()
min_ = train.groupby(['Store'])['Sales'].min()

train['Mean_sales'] = train['Store'].apply(lambda x: mean[x])
train['Median_sales'] = train['Store'].apply(lambda x: median[x])
train['Max_sales'] = train['Store'].apply(lambda x: max_[x])
train['Min_sales'] = train['Store'].apply(lambda x: min_[x])

test['Mean_sales'] = test['Store'].apply(lambda x: mean[x])
test['Median_sales'] = test['Store'].apply(lambda x: median[x])
test['Max_sales'] = test['Store'].apply(lambda x: max_[x])
test['Min_sales'] = test['Store'].apply(lambda x: min_[x])

#lb 1039

In [242]:
# store's mean, max, min sale

mean = train.groupby(['Store'])['Customers'].mean()
median = train.groupby(['Store'])['Customers'].median()
max_ = train.groupby(['Store'])['Customers'].max()
min_ = train[train['Customers']!=0].groupby(['Store'])['Customers'].min()

train['Mean_customers'] = train['Store'].apply(lambda x: mean[x])
train['Median_customers'] = train['Store'].apply(lambda x: median[x])
train['Max_customers'] = train['Store'].apply(lambda x: max_[x])
train['Min_customers'] = train['Store'].apply(lambda x: min_[x])

test['Mean_customers'] = test['Store'].apply(lambda x: mean[x])
test['Median_customers'] = test['Store'].apply(lambda x: median[x])
test['Max_customers'] = test['Store'].apply(lambda x: max_[x])
test['Min_customers'] = test['Store'].apply(lambda x: min_[x])

#lb 

In [243]:
# Sales/mean,median customers

train['Mean_Sales_/_Customer'] = train['Mean_sales'] / train['Mean_customers']
train['Median_Sales_/_Customer'] = train['Median_sales'] / train['Median_customers']
test['Mean_Sales_/_Customer'] = test['Mean_sales'] / test['Mean_customers']
test['Median_Sales_/_Customer'] = test['Median_sales'] / test['Median_customers']

In [244]:
# Store's Sale Per day_of_week

week_day_mean = train.groupby(['Store', 'DayOfWeek'])['Sales'].mean().to_dict()
week_day_median = train.groupby(['Store', 'DayOfWeek'])['Sales'].median().to_dict()
week_day_min = train.groupby(['Store', 'DayOfWeek'])['Sales'].min().to_dict()
week_day_max = train.groupby(['Store', 'DayOfWeek'])['Sales'].max().to_dict()

train['day_of_week_Sale_mean'] = train.apply(lambda x: week_day_mean[(x['Store'], x['DayOfWeek'])], axis = 1)
test['day_of_week_Sale_mean'] = test.apply(lambda x: week_day_mean[(x['Store'], x['DayOfWeek'])], axis = 1)

train['day_of_week_Sale_median'] = train.apply(lambda x: week_day_median[(x['Store'], x['DayOfWeek'])], axis = 1)
test['day_of_week_Sale_median'] = test.apply(lambda x: week_day_median[(x['Store'], x['DayOfWeek'])], axis = 1)

train['day_of_week_Sale_min'] = train.apply(lambda x: week_day_min[(x['Store'], x['DayOfWeek'])], axis = 1)
test['day_of_week_Sale_min'] = test.apply(lambda x: week_day_min[(x['Store'], x['DayOfWeek'])], axis = 1)

train['day_of_week_Sale_max'] = train.apply(lambda x: week_day_max[(x['Store'], x['DayOfWeek'])], axis = 1)
test['day_of_week_Sale_max'] = test.apply(lambda x: week_day_max[(x['Store'], x['DayOfWeek'])], axis = 1)


In [245]:
# Store's Sale Per day_of_week and promo

week_day_mean = train.groupby(['Store', 'DayOfWeek', 'Promo'])['Sales'].mean().to_dict()
week_day_median = train.groupby(['Store', 'DayOfWeek', 'Promo'])['Sales'].median().to_dict()
week_day_min = train.groupby(['Store', 'DayOfWeek', 'Promo'])['Sales'].min().to_dict()
week_day_max = train.groupby(['Store', 'DayOfWeek', 'Promo'])['Sales'].max().to_dict()

train['day_of_week_Sale_mean_promo'] = train.apply(lambda x: week_day_mean[(x['Store'], x['DayOfWeek'], x['Promo'])], axis = 1)
test['day_of_week_Sale_mean_promo'] = test.apply(lambda x: week_day_mean[(x['Store'], x['DayOfWeek'], x['Promo'])], axis = 1)

train['day_of_week_Sale_median_promo'] = train.apply(lambda x: week_day_median[(x['Store'], x['DayOfWeek'], x['Promo'])], axis = 1)
test['day_of_week_Sale_median_promo'] = test.apply(lambda x: week_day_median[(x['Store'], x['DayOfWeek'], x['Promo'])], axis = 1)

train['day_of_week_Sale_min_promo'] = train.apply(lambda x: week_day_min[(x['Store'], x['DayOfWeek'], x['Promo'])], axis = 1)
test['day_of_week_Sale_min_promo'] = test.apply(lambda x: week_day_min[(x['Store'], x['DayOfWeek'], x['Promo'])], axis = 1)

train['day_of_week_Sale_max_promo'] = train.apply(lambda x: week_day_max[(x['Store'], x['DayOfWeek'], x['Promo'])], axis = 1)
test['day_of_week_Sale_max_promo'] = test.apply(lambda x: week_day_max[(x['Store'], x['DayOfWeek'], x['Promo'])], axis = 1)


In [247]:
# Dropping Samples with Open=0

index = train[ train['Open'] == 0].index
train.drop(index, inplace = True, axis = 0)

# Store's Sale Per day_of_week and promo

week_day_mean = train.groupby(['Store', 'week'])['Sales'].mean().to_dict()
week_day_median = train.groupby(['Store', 'week'])['Sales'].median().to_dict()
week_day_min = train.groupby(['Store', 'week'])['Sales'].min().to_dict()
week_day_max = train.groupby(['Store', 'week'])['Sales'].max().to_dict()

def imputer(d, x):
    try:
        val = d[(x['Store'], x['week'])]
        return val
    except:
        return None


train['week_Sale_mean'] = train.apply(lambda x: imputer(week_day_mean, x), axis = 1)
test['week_Sale_mean'] = test.apply(lambda x: imputer(week_day_mean, x), axis = 1)

train['week_Sale_median'] = train.apply(lambda x: imputer(week_day_median, x), axis = 1)
test['week_Sale_median'] = test.apply(lambda x: imputer(week_day_median, x), axis = 1)

train['week_Sale_min'] = train.apply(lambda x: imputer(week_day_min, x), axis = 1)
test['week_Sale_min'] = test.apply(lambda x: imputer(week_day_min, x), axis = 1)

train['week_Sale_max'] = train.apply(lambda x: imputer(week_day_max, x), axis = 1)
test['week_Sale_max'] = test.apply(lambda x: imputer(week_day_max, x), axis = 1)


In [262]:
train['Sale/Cus'] = train['Sales']/train['Customers']

train_mean = train.groupby(['Store'])['Sale/Cus'].mean().to_dict()
train_median = train.groupby(['Store'])['Sale/Cus'].median().to_dict()
train_min = train.groupby(['Store'])['Sale/Cus'].min().to_dict()
train_max = train.groupby(['Store'])['Sale/Cus'].max().to_dict()

test['Sale/Cus'] = test['Store'].apply(lambda x: train_mean[x])
test['Sale/Cus'] = test['Store'].apply(lambda x: train_median[x])
test['Sale/Cus'] = test['Store'].apply(lambda x: train_min[x])
test['Sale/Cus'] = test['Store'].apply(lambda x: train_max[x])

In [263]:
features = [col for col in train.columns if col not in [target, date, customer]]

trn, val = train_test_split(train, test_size = 0.2, random_state = 1999)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test[features]

In [279]:
%%time
lr = XGBRegressor(random_state = 1,tree_method = 'gpu_hist',n_estimators = 1000, max_depth = 6, learning_rate = 0.3)
lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error((y_val)**2, (preds)**2))

print(f'mean_squared_log_error is : {error}')

mean_squared_log_error is : 565.7701377090194
Wall time: 23 s


In [268]:
%%time

#lr = XGBRegressor(random_state=1, tree_method = 'gpu_hist', n_estimators = 1000,
#                  max_depth = 7, learning_rate = 0.3)

#lr = XGBRegressor(random_state = 1, tree_method = 'gpu_hist', n_estimators = 500, learning_rate = 0.5, max_depth = 8)
lr = XGBRegressor(random_state = 1,tree_method = 'gpu_hist',n_estimators = 500)
lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error((y_val)**2, (preds)**2))

print(f'mean_squared_log_error is : {error}')

# lr = LGBMRegressor(random_state=1)

# lr.fit(X_trn, y_trn)

# preds = lr.predict(X_val)
# preds = np.abs(preds)

# error = np.sqrt(mean_squared_error((y_val)**2, (preds)**2))

# print(f'mean_squared_log_error is : {error}')
#645

mean_squared_log_error is : 584.357366063345
Wall time: 12 s


In [282]:
#xgb = XGBRegressor(random_state=1, max_depth = 7, n_estimators = 1200, learning_rate = 0.01, subsample=0.9, min_child_weight = 13, tree_method='gpu_hist')
# xgb = XGBRegressor(random_state=1, tree_method = 'gpu_hist', n_estimators = 1000,
#                   max_depth = 7, learning_rate = 0.3)

# xgb = XGBRegressor(random_state = 1, tree_method = 'gpu_hist', n_estimators = 500,
#                    learning_rate = 0.5, max_depth = 8)
xgb = XGBRegressor(random_state = 1,tree_method = 'gpu_hist',n_estimators = 1000, max_depth = 6, learning_rate = 0.3)

In [294]:
# Cross Validation for Boosting
def cross_val(regressor, train, test, features, name):
    N_splits = 5
    
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    target_col = train[target]
    
    folds = StratifiedKFold(n_splits = N_splits, shuffle = True,random_state = 1999)
    stratified_target = pd.qcut( train[target], 10, labels=False, duplicates='drop')
    for index, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
        print(f'\n================================Fold{index + 1}===================================')
        
        #### Train Set
        X_trn, y_trn = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
        
        #### Validation Set
        X_val, y_val = train[features].iloc[val_idx], train[target].iloc[val_idx]
        
        #### Test Set
        X_test = test[features]
        
        ############ Fitting #############
        _ = regressor.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 50, verbose = 100, eval_metric='rmse')
        
        ############ Predicting #############
        val_preds = np.abs(regressor.predict(X_val))
        test_preds = np.abs(regressor.predict(X_test))
        
        error = np.sqrt(mean_squared_error((y_val)**2, (val_preds)**2))
        print(f'\n Root Mean Squared Error for Validation set is : {error}')
        
        oofs[val_idx] = val_preds
        preds += test_preds / N_splits
        
    total_error = np.sqrt(mean_squared_error((target_col)**2, (oofs)**2))
    print(f'\n\Root Mean Squared Error for oofs is {total_error}')
    
    return oofs, preds

In [295]:
%%time
xgb_oofs, xgb_preds = cross_val(xgb, train, test, features, 'xgb')


================================Fold1===================================
[0]	validation_0-rmse:58.75579
[100]	validation_0-rmse:3.59620
[200]	validation_0-rmse:3.43378
[300]	validation_0-rmse:3.36143
[400]	validation_0-rmse:3.30277
[500]	validation_0-rmse:3.26603
[600]	validation_0-rmse:3.24284
[700]	validation_0-rmse:3.21943
[800]	validation_0-rmse:3.20555
[900]	validation_0-rmse:3.19347
[999]	validation_0-rmse:3.18298

 Root Mean Squared Error for Validation set is : 572.7916291865711

================================Fold2===================================
[0]	validation_0-rmse:58.76134
[100]	validation_0-rmse:3.59801
[200]	validation_0-rmse:3.45367
[300]	validation_0-rmse:3.37943
[400]	validation_0-rmse:3.32844
[500]	validation_0-rmse:3.29058
[600]	validation_0-rmse:3.26610
[700]	validation_0-rmse:3.24468
[800]	validation_0-rmse:3.22613
[900]	validation_0-rmse:3.21452
[999]	validation_0-rmse:3.20239

 Root Mean Squared Error for Validation set is : 572.6782350754131

=============

In [285]:
# %%time
# lr = XGBRegressor(random_state=1, n_jobs = -1)

# lr.fit(train[features], train[target])

# preds = lr.predict(test[features])
# preds = np.abs(preds)
# # With eval set and no cat_cols: lb score 1093
#xgb_preds = xgb_preds**2

In [298]:
test['Predictions'] = xgb_preds**2
test.loc[ test['Open'] == 0, 'Predictions'] = 0
preds = test['Predictions']

index = [i for i in range(test.shape[0])]

d = list(zip(index, preds))

ss = pd.DataFrame(d, columns = ['index', 'Sales'])

In [299]:
ss.to_csv(path + "\\new_feats.csv", index = False)

In [300]:
ss

,index,Sales
0,0,0.000000
1,1,3378.579705
2,2,4610.134875
3,3,3580.485045
4,4,4726.274411
...,...,...
305158,305158,9508.029039
305159,305159,4600.701374
305160,305160,3905.258718
305161,305161,2588.719775
